In [1]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from FeatureEngineering import create_time_matrix
from MatchingAlgorithm import run_algorithm

In [6]:
catchment = pd.read_csv("Simulated Data/catchment_data_1576.csv", dtype={'lat': str, 'long': str})
catchment['Address'] = catchment['lat'] + "," + catchment['long']

orders = pd.read_csv("Simulated Data/order_data_1576.csv", dtype={'long': str, 'lat': str})
all_columns = orders.columns
service_cols = all_columns[all_columns.str.contains('service')]
orders = orders.iloc[0:23]
orders['Address'] = orders['lat'].str.cat(orders['long'], sep=',')

supply_df = pd.read_csv("Simulated Data/phleb_data_1576.csv", dtype={'home_lat': str, 'home_long': str})
phleb = supply_df
phleb = phleb.iloc[0:3]
phleb['Address'] = phleb['home_lat'].str.cat(phleb['home_long'], sep=',')

inverse_ratings = 5 - phleb['service_rating']
inverse_ratings

order_window = [(6 * 60, 18 * 60)] #ending depot
order_window.extend([(int(start) * 60, (int(start)+1) * 60) for start in phleb['shift_start']])
order_window.extend([(int(start) * 60, int(start+1) * 60) for start in orders['order_start']])

numPhleb = phleb.shape[0]

revenues = [1] #ending depot
revenues.extend([1 for _ in range(numPhleb)])
revenues.extend(orders['price'])

API_key = '' #INPUT YOUR OWN API KEY

#Add Catchment Area first
addresses_list = []
addresses_list.append(catchment['Address'].iloc[0])

#Phlebotomists' Starting Points first
addresses_list.extend(phleb['Address'])
#Orders' Locations
addresses_list.extend(orders['Address'])

time_matrix = create_time_matrix(addresses_list, API_key)

servicing_times = [0]
servicing_times.extend([0 for _ in range(numPhleb)])
servicing_times.extend(orders['duration'] + orders['buffer'])

#Service-Expertise Constraints
def find_applicable_exp(row):
    args = np.empty(0)
    for val in row:
        args = np.append(args, val)
    idx = [args == 1]
    service_needs = service_cols[idx[0]]

    expertiseName = "expertise_{}".format(service_needs[0].split("_")[1])
    temp = phleb.loc[(phleb[expertiseName] == 1)]

    if len(service_needs) > 1:
        for service in service_needs[1:]:
            expertiseName = "expertise_{}".format(service.split("_")[1])
            temp = temp.loc[(temp[expertiseName] == 1)]
    
    return temp.index.to_list()

orders['Acceptable Phleb Indices'] = orders[service_cols].apply(find_applicable_exp, axis=1)
expertises = [1] #ending depot
expertises.extend([1 for _ in range(numPhleb)])
expertises.extend(orders['Acceptable Phleb Indices'])

order_ids = ["NA"] #ending depot
order_ids.extend(["NA" for _ in range(numPhleb)])
order_ids.extend(orders['order_id'])
locations_metadata = zip(addresses_list, order_ids)

metadata = {'Locations': [{"Location Index": idx, "Coordinate": metadata[0], "Order Id": str(metadata[1])}for idx, metadata in enumerate(locations_metadata)]}
metadata['Phlebotomists'] = [{"Phlebotomist Index": idx, "Id": id}for idx, id in enumerate(phleb['phleb_id'])]

C:\Users\sisda\AppData\Local\Temp\ipykernel_18688\224595809.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phleb['Address'] = phleb['home_lat'].str.cat(phleb['home_long'], sep=',')


In [7]:
result = run_algorithm(time_matrix, order_window, revenues, numPhleb, servicing_times, expertises, inverse_ratings, metadata)

In [25]:
json_object = json.loads(result)
routes = json_object['Routes']
routes = pd.json_normalize(routes)
routes

,Phlebotomist Index,Printable Route,Total Travel Time,Total Loads,Locations Sequence,Start Times Sequence,End Times Sequence,Slack Times Sequence
0,0,Route for Phlebotomist 0:\nLocation 1 Start(42...,122,9,"[1, 26, 11, 6, 10, 15, 21, 13, 4, 7, 0]","[[420, 420], [430, 445], [496, 500], [546, 550...","[[420, 420], [471, 486], [536, 540], [572, 576...","[[0, 15], [0, 15], [0, 4], [0, 4], [2, 29], [0..."
1,1,Route for Phlebotomist 1:\nLocation 2 Start(42...,87,6,"[2, 23, 24, 12, 5, 8, 18, 0]","[[420, 420], [430, 432], [463, 465], [524, 525...","[[420, 420], [456, 458], [507, 509], [580, 581...","[[0, 2], [0, 2], [0, 2], [0, 1], [192, 193], [..."
2,2,Route for Phlebotomist 2:\nLocation 3 Start(42...,68,7,"[3, 19, 9, 16, 14, 17, 20, 0]","[[420, 420], [448, 465], [509, 526], [558, 575...","[[420, 420], [505, 522], [553, 570], [584, 601...","[[15, 32], [0, 17], [0, 17], [0, 17], [3, 20],..."


In [20]:
test = json.loads(result)
test = pd.json_normalize(test)
test

,Routes,Metadata.Locations,Metadata.Phlebotomists,Model.Objective Number,Model.Status,Model.Total Revenue Lost,Model.Total Number of Nodes Dropped,Model.Nodes Dropped,Model.Revenues Dropped,Model.Total Travel Time,Model.Total Loads
0,"[{'Phlebotomist Index': 0, 'Printable Route': ...","[{'Location Index': 0, 'Coordinate': '28.48431...","[{'Phlebotomist Index': 0, 'Id': 0}, {'Phlebot...",69496,1,51.544838,2,"[22, 25]","[34.36322532027129, 17.181612660135645]",277,22


In [8]:
import json

def get_vacant_time_windows(matching_algo_results):
    json_object = json.loads(matching_algo_results)
    data = []
    routes = json_object['Routes']

    for phleb in range(len(routes)):
        curr_phleb = routes[phleb]
        phleb_index = curr_phleb['Phlebotomist Index']
        slack_times = curr_phleb['Slack Times Sequence']
        locations = curr_phleb['Locations Sequence']
        start_times = curr_phleb['Start Times Sequence']
        end_times = curr_phleb['End Times Sequence']
        for index in range(len(slack_times)):
            slack_time = slack_times[index]
            slack_time_min = slack_time[0]
            slack_time_max = slack_time[1]
            if slack_time_max == 0: #no slack time between this loc and the next
                continue
            else:
                # Case 1: Phlebotomist ends his/her service at the earliest possible time. Slack time before next loc should be max
                # Phlebotomist immediately becomes available at his/her current location
                temp_data = []
                temp_data.append(phleb_index)
                temp_data.append(end_times[index][0]) # Time window start  - take earliest possible time
                temp_data.append(end_times[index][0] + slack_time_max) # Time window end. Adding the max slack time
                temp_data.append(slack_time_max)
                temp_data.append(locations[index]) # Last location before free
                temp_data.append(locations[index + 1]) # First location after free
                data.append(temp_data)

                # Case 2: Phlebotomist ends his/her service at the latest possible time. Slack time before next loc should be min
                # Phlebotomist immediately becomes available at his/her current location
                ### CASE 2 OMITTED DUE TO IT BEING SUBSET OF CASE 1 (can uncomment this chunk to visualize if needed) ###
                '''
                if index != 0:
                    temp_data = []
                    temp_data.append(phleb_index)
                    temp_data.append(end_times[index][1]) # Time window start - take latest possible time
                    temp_data.append(end_times[index][1] + slack_time_min) # Time window end. Adding the min slack time
                    temp_data.append(slack_time_min)
                    temp_data.append(locations[index]) # Last location before free
                    temp_data.append(locations[index + 1]) # First location after free
                    data.append(temp_data)
                '''

                # Case 3: Phlebotomist ends his/her service at the earliest possible time. Slack time before next loc should be max
                # Phlebotomist immediately travels to his/her next location, and becomes available there
                temp_data = []
                temp_data.append(phleb_index)
                travel_time = start_times[index + 1][0] - end_times[index][1] - slack_time_min
                temp_data.append(end_times[index][0] + travel_time) # Time window start - take earliest possible time + travel time to next loc
                temp_data.append(start_times[index + 1][1]) # Time window end, which is basically the latest possible start time of the next loc (i.e. the loc that the phlebo has traveled to)
                temp_data.append(start_times[index + 1][1] - end_times[index][0] - travel_time)
                temp_data.append(locations[index + 1]) # Last location before free
                temp_data.append(locations[index + 1]) # First location after free
                data.append(temp_data)

                # Case 4: Phlebotomist ends his/her service at the latest possible time. Slack time before next loc should be min
                # Phlebotomist immediately travels to his/her next location, and becomes available there
                ### CASE 4 OMITTED DUE TO IT BEING SUBSET OF CASE 3 (can uncomment this chunk to visualize if needed) ###
                '''
                if index != 0:
                    temp_data = []
                    temp_data.append(phleb_index)
                    travel_time = start_times[index + 1][0] - end_times[index][1] - slack_time_min
                    temp_data.append(end_times[index][1] + travel_time) # Time window start - take latest possible time + travel time to next loc
                    temp_data.append(start_times[index + 1][1]) # Time window end, which is basically the latest possible start time of the next loc (i.e. the loc that the phlebo has traveled to)
                    temp_data.append('null')
                    temp_data.append(locations[index + 1]) # Last location before free
                    temp_data.append(locations[index + 1]) # First location after free
                    data.append(temp_data)
                '''

    vacant_time_windows = pd.DataFrame(columns=['PhlebotomistIndex', 'TimeWindowStart', 'TimeWindowEnd', 'TimeWindowDuration',
                                                'LastLocBeforeFree', 'FirstLocAfterFree'],
                                      data=data)

    return vacant_time_windows

In [9]:
get_vacant_time_windows(result)

,PhlebotomistIndex,TimeWindowStart,TimeWindowEnd,TimeWindowDuration,LastLocBeforeFree,FirstLocAfterFree
0,0,420,435,15,1,26
1,0,430,445,15,26,26
2,0,471,486,15,26,11
3,0,481,500,19,11,11
4,0,536,540,4,11,6
5,0,542,550,8,6,6
6,0,572,576,4,6,10
7,0,580,588,8,10,10
8,0,640,669,29,10,15
9,0,656,685,29,15,15
